6 September 2019: This notebook serves to investigate the influence of different timesteps, $\Delta t$, in the advection step. The default step used in most particle runs so far is 5 seconds. One run, ``control_y300_P3590x590_S2000-1-1_D90_DT60_ODT12_LAT60.5-89.5_LON-179.5-179.5`` has a time step of 60 seconds. The particle set of this run can be compared to ``pset_control_y300_P3590x590_S2000-1-1_D30_DT5_ODT12_LAT60.5-89.5_LON-179.5-179.5``, since all settings, except for the _runtime_ and $\Delta t$ are equal.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

In [4]:
pset_path = '/data/oceanparcels/output_data/data_Daan/'

In [5]:
ds_dt60 = xr.open_dataset(pset_path+'pset_control_y300_P3590x590_S2000-1-1_D90_DT60_ODT12_LAT60.5-89.5_LON-179.5-179.5.nc')
ds_dt5 = xr.open_dataset(pset_path+'pset_control_y300_P3590x590_S2000-1-1_D30_DT5_ODT12_LAT60.5-89.5_LON-179.5-179.5.nc')

In [6]:
print('Technical data')
print('--------------')
print('-----dt60-----')
print(ds_dt60)
print('-----dt5------')
print(ds_dt5)

Technical data
--------------
-----dt60-----
<xarray.Dataset>
Dimensions:     (obs: 181, traj: 2118100)
Dimensions without coordinates: obs, traj
Data variables:
    trajectory  (traj, obs) int32 ...
    time        (traj, obs) datetime64[ns] ...
    lat         (traj, obs) float32 ...
    lon         (traj, obs) float32 ...
    z           (traj, obs) float32 ...
Attributes:
    feature_type:           trajectory
    Conventions:            CF-1.6/CF-1.7
    ncei_template_version:  NCEI_NetCDF_Trajectory_Template_v2.0
    parcels_version:        2.0.0b3.dev93+g5cdb2e5
    parcels_mesh:           spherical
-----dt5------
<xarray.Dataset>
Dimensions:     (obs: 61, traj: 2118100)
Dimensions without coordinates: obs, traj
Data variables:
    trajectory  (traj, obs) int32 ...
    time        (traj, obs) datetime64[ns] ...
    lat         (traj, obs) float32 ...
    lon         (traj, obs) float32 ...
    z           (traj, obs) float32 ...
Attributes:
    feature_type:           trajectory

# Indices

``ds_dt60`` covers a time span of 90 days:

In [25]:
ds_dt60.time[0,-1].values

numpy.datetime64('2000-03-31T00:00:00.000000000')

``ds_dt5`` only covers a time span of 30 days:

In [26]:
ds_dt5.time[0,-1].values

numpy.datetime64('2000-01-31T00:00:00.000000000')

This means we have to be careful with choosing the correct value in ``ds_dt60`` so we match up our data with ``ds_dt5``. The correct index is ``60``:

In [28]:
ds_dt60.time[0,60].values

numpy.datetime64('2000-01-31T00:00:00.000000000')

# Position comparison
Since both simulations stem from the same original particle grid, we can readily compare the final positions of the particles. This can for example be done for particles released along a certain longitude band/meridian. We investigate $0^\circ$, since the prime meridian coincides with the Fram Straight, and should thus consist of particles in water (as opposed to the trivial case of particles on land).

Note: these simulations were carried out with initial particle grids where particles on land were not yet removed.

## Trajectory index
The initial lat-lon positions are simply stored in sequential order, first going from 179.5W to 179.5E, then going from 60.5N to 89.5N. Therefore, we have to be mindful that when we want to select a meridian, we have to select each 3590th element in our array. ``1795::3590`` selects the prime meridian:

In [58]:
ds_dt60.lon[1795::3590,0]

<xarray.DataArray 'lon' (traj: 590)>
array([0.050014, 0.050014, 0.050014, ..., 0.050014, 0.050014, 0.050014],
      dtype=float32)
Dimensions without coordinates: traj
Attributes:
    long_name:      
    standard_name:  longitude
    units:          degrees_east
    axis:           X

In [60]:
ds_dt5.lon[1795::3590,0]

<xarray.DataArray 'lon' (traj: 590)>
array([0.050014, 0.050014, 0.050014, ..., 0.050014, 0.050014, 0.050014],
      dtype=float32)
Dimensions without coordinates: traj
Attributes:
    long_name:      
    standard_name:  longitude
    units:          degrees_east
    axis:           X

In [59]:
ds_dt60.lat[1795::3590,0]

<xarray.DataArray 'lat' (traj: 590)>
array([60.5     , 60.549236, 60.598473, ..., 89.40153 , 89.45077 , 89.5     ],
      dtype=float32)
Dimensions without coordinates: traj
Attributes:
    long_name:      
    standard_name:  latitude
    units:          degrees_north
    axis:           Y

In [61]:
ds_dt5.lat[1795::3590,0]

<xarray.DataArray 'lat' (traj: 590)>
array([60.5     , 60.549236, 60.598473, ..., 89.40153 , 89.45077 , 89.5     ],
      dtype=float32)
Dimensions without coordinates: traj
Attributes:
    long_name:      
    standard_name:  latitude
    units:          degrees_north
    axis:           Y

## Distances shifted
We can now simply take the euclidian difference between each final location. Note that we use index ``60`` for ``ds_dt60``. First we have to define a function to compute distances using the Haversine formula<sup>[1]</sup>.

[1]: https://medium.com/@petehouston/calculate-distance-of-two-locations-on-earth-using-python-1501b1944d97

In [71]:
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    return 2 * 6371 * np.arcsin(np.sqrt(a))

The rest is straightforward:

In [ ]:
merid_dist_diff = haversine(ds_dt5.lon[1795::3590,:],\
                            ds_dt5.lat[1795::3590,:],\
                            ds_dt60.lon[1795::3590,60],\
                            ds_dt60.lon[1795::3590,60])